In [ ]:
import os
import sys
import cv2
import pandas as pd
import zipfile
import traceback
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
from torchvision import transforms
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

#### Utils file

In [ ]:
import joblib
import yaml
import torch
import torch.nn as nn


def dump(value=None, filename=None):
    if (value is not None) and (filename is not None):
        joblib.dump(value=value, filename=filename)

    else:
        raise ValueError("value and filename must be provided".capitalize())


def load(filename=None):
    if filename is not None:
        return joblib.load(filename=filename)

    else:
        raise ValueError("filename should be defined".capitalize())


def device_init(device="cuda"):
    if device == "cuda":
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")

    elif device == "mps":
        return torch.device("mps" if torch.backends.mps.is_available() else "cpu")

    else:
        return torch.device("cpu")


def weight_init(m):
    classname = m.__class__

    if classname.find("Conv") != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)

    elif classname.find("BatchNorm") != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)


def config():
    with open("./config.yml", "r") as file:
        return yaml.safe_load(file)


class CustomException(Exception):
    def __init__(self, message=None):
        if message is not None:
            return message

In [ ]:
class Loader:
    def __init__(
        self, image_path=None, channels=3, image_size=128, batch_size=1, split_size=0.25
    ):
        self.image_path = image_path
        self.channels = channels
        self.image_size = image_size
        self.batch_size = batch_size
        self.split_size = split_size

        self.X = []
        self.y = []

        self.CONFIG = config()

    def unzip_folder(self):
        if not os.path.exists(self.CONFIG["path"]["PROCESSED_DATA_PATH"]):
            os.makedirs(self.CONFIG["path"]["PROCESSED_DATA_PATH"])

            print(
                "Folder is created successfully in the path of {}".format(
                    self.CONFIG["path"]["PROCESSED_DATA_PATH"]
                )
            )

        with zipfile.ZipFile(self.image_path, "r") as file:
            file.extractall(path=self.CONFIG["path"]["PROCESSED_DATA_PATH"])

        print(
            "Data unzipped successfully and store in the folder of {}".format(
                self.CONFIG["path"]["PROCESSED_DATA_PATH"]
            )
        )

    def transforms(self, type="train"):
        if type == "train":
            return transforms.Compose(
                [
                    transforms.Resize(
                        (self.image_size, self.image_size), Image.BICUBIC
                    ),
                    transforms.ToTensor(),
                    transforms.CenterCrop((self.image_size, self.image_size)),
                    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
                ]
            )
        elif type == "valid":
            return transforms.Compose(
                [
                    transforms.Resize(
                        (self.image_size // 2, self.image_size // 2), Image.BICUBIC
                    ),
                    transforms.ToTensor(),
                    transforms.CenterCrop((self.image_size // 2, self.image_size // 2)),
                    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
                ]
            )

    def split_dataset(self, X, y):
        if isinstance(X, list) and isinstance(y, list):
            X_train, X_test, y_train, y_test = train_test_split(
                X, y, test_size=self.split_size, random_state=42, shuffle=True
            )

            return X_train, X_test, y_train, y_test

        else:
            raise ValueError("X and y should be in the list format".capitalize())

    def feature_extractor(self):
        self.processed_data_path = self.CONFIG["path"]["PROCESSED_DATA_PATH"]

        X = os.path.join(self.processed_data_path, "dataset", "X")
        y = os.path.join(self.processed_data_path, "dataset", "y")

        for image in tqdm(os.listdir(X)):
            if image in os.listdir(y):
                image_X = os.path.join(X, image)
                image_Y = os.path.join(y, image)

                if (image_X is not None) and (image_Y is not None):
                    image_X = cv2.imread(image_X)
                    image_Y = cv2.imread(image_Y)

                    image_X = cv2.cvtColor(image_X, cv2.COLOR_BGR2RGB)
                    image_Y = cv2.cvtColor(image_Y, cv2.COLOR_BGR2RGB)

                    image_X = Image.fromarray(image_X)
                    image_Y = Image.fromarray(image_Y)

                    image_X = self.transforms(type="train")(image_X)
                    image_Y = self.transforms(type="valid")(image_Y)

                    self.X.append(image_X)
                    self.y.append(image_Y)

                else:
                    raise Exception(
                        "Image {} is not found in the path of {}".format(
                            image, self.processed_data_path
                        )
                    )

        try:
            X_train, X_test, y_train, y_test = self.split_dataset(X=self.X, y=self.y)

        except ValueError as e:
            print(e)
            traceback.print_exc()

        except Exception as e:
            print(e)
            traceback.print_exc()

        else:
            return {
                "X_train": X_train,
                "X_test": X_test,
                "y_train": y_train,
                "y_test": y_test,
            }

    def create_dataloader(self):
        dataset = self.feature_extractor()

        train_dataloader = DataLoader(
            dataset=list(zip(dataset["X_train"], dataset["y_train"])),
            batch_size=self.batch_size,
            shuffle=True,
        )
        test_dataloader = DataLoader(
            dataset=list(zip(dataset["X_test"], dataset["y_test"])),
            batch_size=self.batch_size * 8,
            shuffle=True,
        )

        for value, filename in [
            (train_dataloader, "train_dataloader"),
            (test_dataloader, "test_dataloader"),
        ]:
            dump(
                value=value,
                filename=os.path.join(
                    config()["path"]["PROCESSED_DATA_PATH"], filename + ".pkl"
                ),
            )

        print(
            "data is stored in the folder {}".format(
                config()["path"]["PROCESSED_DATA_PATH"]
            )
        )

    @staticmethod
    def dataset_details():
        processed_path = config()["path"]["PROCESSED_DATA_PATH"]
        artifacts_path = config()["path"]["ARTIFACTS_PATH"]

        if os.path.exists(processed_path):
            train_dataloader = load(
                filename=os.path.join(processed_path, "train_dataloader.pkl")
            )
            valid_dataloader = load(
                filename=os.path.join(processed_path, "test_dataloader.pkl")
            )

            train_data, train_label = next(iter(train_dataloader))
            valid_data, valid_label = next(iter(valid_dataloader))

            pd.DataFrame(
                {
                    "total_data_points": (sum(X.size(0) for X, _ in train_dataloader))
                    + sum(X.size(0) for X, _ in valid_dataloader),
                    "train_data_points": sum(X.size(0) for X, _ in train_dataloader),
                    "valid_data_points": sum(X.size(0) for X, _ in valid_dataloader),
                    "train_image_size(X)": str(train_data.size()),
                    "valid_image_size(X)": str(valid_data.size()),
                    "train_image_size(y)": str(train_label.size()),
                    "valid_image_size(y)": str(valid_label.size()),
                },
                index=["Quantity"],
            ).to_csv(os.path.join(artifacts_path, "dataset_details.csv"))

    @staticmethod
    def plot_images():
        processed_path = config()["path"]["PROCESSED_DATA_PATH"]
        artifacts_path = config()["path"]["ARTIFACTS_PATH"]

        train_dataloader = load(os.path.join(processed_path, "test_dataloader.pkl"))
        data, label = next(iter(train_dataloader))

        number_of_rows = data.size(0) // 2
        number_of_columns = data.size(0) // number_of_rows

        print(number_of_rows, number_of_columns)

        plt.figure(figsize=(20, 10))

        for index, image in enumerate(data):
            X = image.permute(1, 2, 0).detach().numpy()
            X = (X - X.min()) / (X.max() - X.min())

            y = label[index].permute(1, 2, 0).detach().numpy()
            y = (y - y.min()) / (y.max() - y.min())

            plt.subplot(2 * number_of_rows, 2 * number_of_columns, 2 * index + 1)
            plt.imshow(X)
            plt.title("X")
            plt.axis("off")

            plt.subplot(2 * number_of_rows, 2 * number_of_columns, 2 * index + 2)
            plt.imshow(y)
            plt.title("y")
            plt.axis("off")

        plt.tight_layout()
        plt.savefig(os.path.join(artifacts_path, "images.png"))
        plt.show()

        print("Image is saved in the folder of {}".format(artifacts_path))


if __name__ == "__main__":
    loader = Loader(
        image_path="./data/raw/dataset.zip",
        channels=3,
        batch_size=1,
        split_size=0.10,
    )

    loader.unzip_folder()
    loader.feature_extractor()
    loader.create_dataloader()
    Loader.dataset_details()
    Loader.plot_images()

#### Encoder Block

In [ ]:
import torch
import torch.nn as nn

In [ ]:
class EncoderBlock(nn.Module):
    def __init__(self, in_channels = 3, out_channels = 64, kernel_size = 4, stride = 2, padding = 1, use_norm = True):
        super(EncoderBlock, self).__init__()
        
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.is_normalization = use_norm
        
        self.encoder_block = self.block()
        
    def block(self):
        self.layers = []
        
        self.layers.append(
            nn.Conv2d(
                in_channels=self.in_channels,
                out_channels=self.out_channels,
                kernel_size=self.kernel_size,
                stride=self.stride,
                padding=self.padding
            )
        )
        
        if self.is_normalization:
            self.layers.append(nn.BatchNorm2d(num_features=self.out_channels))
            
        self.layers.append(nn.LeakyReLU(negative_slope=0.2, inplace=True))
        
        return nn.Sequential(*self.layers)
    
    def forward(self, x):
        if isinstance(x, torch.Tensor):
            return self.encoder_block(x)
        else:
            raise ValueError("Input should be in the format of the tensor".capitalize())


if __name__ == "__main__":
    in_channels = 3
    out_channels = 64
    kernel_size = 4
    stride = 2
    padding = 1

    layers = []

    for _ in range(2):
        layers.append(EncoderBlock(
            in_channels=in_channels, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding, use_norm=False))

        in_channels = out_channels

    for _ in range(3):
        layers.append(EncoderBlock(
            in_channels=in_channels, out_channels=out_channels * 2, kernel_size=kernel_size, stride=stride, padding=padding, use_norm=True))

        in_channels = out_channels * 2
        out_channels = in_channels

    layers.append(
        nn.Conv2d(
            in_channels=in_channels,
            out_channels=4000,
            kernel_size=1,
            stride=1,
            padding=0
        )
    ) 

    model = nn.Sequential(*layers)

    assert model(torch.randn(1, 3, 128, 128)).size() == (1, 4000, 4, 4)

#### Decoder Block

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(
        self, in_channels=4000, out_channels=512, kernel_size=4, stride=2, padding=1
    ):
        super(DecoderBlock, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding

        self.decoder_block = self.block()

    def block(self):
        self.layers = []

        self.layers.append(
            nn.ConvTranspose2d(
                in_channels=self.in_channels,
                out_channels=self.out_channels,
                kernel_size=self.kernel_size,
                stride=self.stride,
                padding=self.padding,
            )
        )

        self.layers.append(nn.BatchNorm2d(num_features=self.out_channels))
        self.layers.append(nn.ReLU(inplace=True))

        return nn.Sequential(*self.layers)

    def forward(self, x):
        if isinstance(x, torch.Tensor):
            return self.decoder_block(x)

        else:
            raise ValueError("Input should be in the format of the tensor".capitalize())


if __name__ == "__main__":
    layers = []

    in_channels = 4000
    out_channels = 512
    kernel_size = 4
    stride = 2
    padding = 1

    for _ in range(4):
        layers.append(
            DecoderBlock(
                in_channels=in_channels,
                out_channels=out_channels,
                kernel_size=kernel_size,
                stride=stride,
                padding=padding,
            )
        )
        in_channels = out_channels
        out_channels = in_channels // 2

    layers.append(
        nn.ConvTranspose2d(
            in_channels=in_channels,
            out_channels=3,
            kernel_size=kernel_size - 1,
            stride=stride // stride,
            padding=padding,
        )
    )

    model = nn.Sequential(*layers)

    print(model(torch.randn(1, 4000, 4, 4)).size())

#### Generator

In [ ]:
class Generator(nn.Module):
    def __init__(self, in_channels=3, out_channels=64):
        super(Generator, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.kernel_size = 4
        self.stride = 2
        self.padding = 1

        self.layers = []

        for _ in range(2):
            self.layers.append(
                EncoderBlock(
                    in_channels=in_channels,
                    out_channels=out_channels,
                    kernel_size=self.kernel_size,
                    stride=self.stride,
                    padding=self.padding,
                    use_norm=False,
                )
            )

            in_channels = out_channels

        for _ in range(3):
            self.layers.append(
                EncoderBlock(
                    in_channels=in_channels,
                    out_channels=out_channels * 2,
                    kernel_size=self.kernel_size,
                    stride=self.stride,
                    padding=self.padding,
                    use_norm=True,
                )
            )

            in_channels = out_channels * 2
            out_channels = in_channels

        self.layers.append(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=4000,
                kernel_size=self.kernel_size // self.kernel_size,
                stride=self.stride // self.stride,
                padding=0,
            )
        )

        in_channels = 4000

        for _ in range(4):
            self.layers.append(
                DecoderBlock(
                    in_channels=in_channels,
                    out_channels=out_channels,
                    kernel_size=self.kernel_size,
                    stride=self.stride,
                    padding=self.padding,
                )
            )
            in_channels = out_channels
            out_channels = in_channels // 2

        self.layers.append(
            nn.ConvTranspose2d(
                in_channels=in_channels,
                out_channels=3,
                kernel_size=self.kernel_size - 1,
                stride=self.stride // self.stride,
                padding=self.padding,
            )
        )

        self.model = nn.Sequential(*self.layers)

    def forward(self, x):
        if isinstance(x, torch.Tensor):
            return self.model(x)
        else:
            raise ValueError("Input should be in the format of the tensor".capitalize())


if __name__ == "__main__":
    netG = Generator()

    print(netG(torch.randn(1, 3, 128, 128)).size())

#### Discriminator Block

In [ ]:
class DiscriminatorBlock(nn.Module):

    def __init__(
        self,
        in_channels=3,
        out_channels=64,
        momentum=0.8,
        use_normalization=True,
        stride=2,
    ):
        super(DiscriminatorBlock, self).__init__()

        self.in_channels = in_channels
        self.out_channels = out_channels

        self.kernel_size = 3
        self.stride = stride
        self.padding = 1
        self.slope = 0.2

        self.momentum = momentum

        self.is_normalization = use_normalization

        self.discriminator_block = self.block()

    def block(self):
        self.layers = []

        self.layers.append(
            nn.Conv2d(
                in_channels=self.in_channels,
                out_channels=self.out_channels,
                kernel_size=self.kernel_size,
                stride=self.stride,
                padding=self.padding,
            )
        )

        if self.is_normalization:
            self.layers.append(
                nn.InstanceNorm2d(
                    num_features=self.out_channels, momentum=self.momentum
                )
            )

        self.layers.append(nn.LeakyReLU(negative_slope=self.slope, inplace=True))

        self.model = nn.Sequential(*self.layers)

    def forward(self, x):
        if isinstance(x, torch.Tensor):
            return self.model(x)

        else:
            raise ValueError("Input should be in the format of the tensor".capitalize())


if __name__ == "__main__":

    layers = []

    in_channels = 3
    out_channels = 54
    kernel_size = 3
    stride = 2
    padding = 1

    for idx in range(4):
        layers.append(
            DiscriminatorBlock(
                in_channels=in_channels,
                out_channels=out_channels,
                stride=1 if (idx + 1) == 4 else 2,
                use_normalization=False if idx == 0 else True,
            )
        )
        in_channels = out_channels
        out_channels *= 2

    layers.append(
        nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=in_channels // in_channels,
                kernel_size=kernel_size,
                stride=stride // stride,
                padding=padding,
            )
        )
    )

    model = nn.Sequential(*layers)

    assert model(torch.randn(1, 3, 64, 64)).size() == (1, 1, 8, 8)

#### Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self, in_channels=3):
        super(Discriminator, self).__init__()

        self.in_channels = in_channels

        self.out_channels = 64
        self.kernel_size = 3
        self.stride = 2
        self.padding = 1

        self.layers = []

        for idx in range(4):
            self.layers.append(
                DiscriminatorBlock(
                    in_channels=self.in_channels,
                    out_channels=self.out_channels,
                    stride=1 if (idx + 1) == 4 else 2,
                    use_normalization=False if idx == 0 else True,
                )
            )
            self.in_channels = self.out_channels
            self.out_channels *= 2

        self.layers.append(
            nn.Sequential(
                nn.Conv2d(
                    in_channels=self.in_channels,
                    out_channels=self.in_channels // self.in_channels,
                    kernel_size=self.kernel_size,
                    stride=self.stride // self.stride,
                    padding=self.padding,
                )
            )
        )

        self.model = nn.Sequential(*self.layers)

    def forward(self, x):
        if isinstance(x, torch.Tensor):
            return self.model(x)

        else:
            raise ValueError("Input should be in the format of the tensor".capitalize())


if __name__ == "__main__":
    netD = Discriminator()

    print(netD)

### Loss

In [ ]:
import torch
import torch.nn as nn


class AdversarialLoss(nn.Module):
    def __init__(self, reduction="mean"):
        super(AdversarialLoss, self).__init__()

        self.reduction = reduction

    def forward(self, pred, actual):
        if (isinstance(pred, torch.Tensor)) and (isinstance(actual, torch.Tensor)):
            self.loss = nn.MSELoss(reduction=self.reduction)

            return self.loss(predicted, actual)

        else:
            raise ValueError(
                "Pred and actual should be in the tensor format".capitalize()
            )


if __name__ == "__main__":

    loss = AdversarialLoss()

    predicted = torch.tensor([1.0, 0.0, 1.0, 0.0])
    actual = torch.tensor([1.0, 0.0, 1.0, 0.0])

    assert loss(predicted, actual) == (0.0)

In [ ]:
import torch
import torch.nn as nn


class PixelLoss(nn.Module):
    def __init__(self, reduction="mean"):
        super(PixelLoss, self).__init__()

        self.reduction = reduction

    def forward(self, pred, actual):
        if isinstance(pred, torch.Tensor) and isinstance(actual, torch.Tensor):
            self.loss = nn.L1Loss(reduction=self.reduction)

            return self.loss(pred, actual)

        else:
            raise ValueError(
                "Pred and actual should be in the tensor format".capitalize()
            )


if __name__ == "__main__":
    loss = PixelLoss()

    predicted = torch.tensor([1.0, 0.0, 1.0, 0.0])
    actual = torch.tensor([1.0, 0.0, 1.0, 0.0])

    assert loss(predicted, actual) == (0.0)

#### Helper

In [ ]:
import traceback
import torch.optim as optim


def load_dataloader():
    processed_path = config()["path"]["PROCESSED_DATA_PATH"]

    if os.path.exists(processed_path):
        train_dataloader = load(os.path.join(processed_path, "train_dataloader.pkl"))
        valid_dataloader = load(os.path.join(processed_path, "test_dataloader.pkl"))

        return {
            "train_dataloader": train_dataloader,
            "valid_dataloader": valid_dataloader,
        }
    else:
        raise CustomException("Cannot import the dataloader".capitalize())
    


def helpers(**kwargs):
    adam = kwargs["adam"]
    SGD = kwargs["SGD"]
    beta1 = kwargs["beta1"]
    beta2 = kwargs["beta2"]
    momentum = kwargs["momentum"]
    lr = kwargs["lr"]

    try:
        netG = Generator(in_channels=3, out_channels=64)
    except Exception as e:
        print("An error is occured {}".format(e))
        traceback.print_exc()

    try:
        netD = Discriminator(in_channels=3)
    except Exception as e:
        print("An error is occured {}".format(e))
        traceback.print_exc()

    if adam:
        optimizerG = optim.Adam(params=netG.parameters(), lr=lr, betas=(beta1, beta2))
        optimizerD = optim.Adam(params=netD.parameters(), lr=lr, betas=(beta1, beta2))

    if SGD:
        optimizerG = optim.SGD(params=netG.parameters(), lr=lr, momentum=momentum)
        optimizerD = optim.SGD(params=netD.parameters(), lr=lr, momentum=momentum)

    adversarial_loss = AdversarialLoss(reduction="mean")
    pixelwise_loss = PixelLoss(reduction="mean")

    try:
        dataloader = load_dataloader()

    except CustomException as e:
        print("An error is occured {}".format(e))
        traceback.print_exc()

    except Exception as e:
        print("An error is occured {}".format(e))
        traceback.print_exc()

    return {
        "train_dataloader": dataloader["train_dataloader"],
        "valid_dataloader": dataloader["valid_dataloader"],
        "netG": netG,
        "netD": netD,
        "optimizerG": optimizerG,
        "optimizerD": optimizerD,
        "adversarial_loss": adversarial_loss,
        "pixelwise_loss": pixelwise_loss,
    }


if __name__ == "__main__":
    init = helpers(
        adam=True, SGD=False, beta1=0.5, beta2=0.999, momentum=0.9, lr=0.0002
    )

    assert init["netG"].__class__.__name__ == "Generator"
    assert init["netD"].__class__.__name__ == "Discriminator"

    assert init["optimizerG"].__class__.__name__ == "Adam"
    assert init["optimizerD"].__class__.__name__ == "Adam"

    assert init["adversarial_loss"].__class__.__name__ == "AdversarialLoss"
    assert init["pixelwise_loss"].__class__.__name__ == "PixelLoss"

#### Unittest

In [ ]:
import sys
import torch
import torch.optim as optim
import torch.nn as nn
import unittest

class UnitTest(unittest.TestCase):
    def setUp(self):
        self.encoder = EncoderBlock()
        self.decoder = DecoderBlock()
        self.netG = Generator()
        self.netD = Discriminator()
        self.init = helpers(
            adam=True, SGD=False, beta1=0.5, beta2=0.999, momentum=0.9, lr=0.0002
        )

    def test_encoder_block(self):
        in_channels = 3
        out_channels = 64
        kernel_size = 4
        stride = 2
        padding = 1

        layers = []

        for _ in range(2):
            layers.append(
                EncoderBlock(
                    in_channels=in_channels,
                    out_channels=out_channels,
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=padding,
                    use_norm=False,
                )
            )

            in_channels = out_channels

        for _ in range(3):
            layers.append(
                EncoderBlock(
                    in_channels=in_channels,
                    out_channels=out_channels * 2,
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=padding,
                    use_norm=True,
                )
            )

            in_channels = out_channels * 2
            out_channels = in_channels

        layers.append(
            nn.Conv2d(
                in_channels=in_channels,
                out_channels=4000,
                kernel_size=kernel_size // kernel_size,
                stride=stride // stride,
                padding=0,
            )
        )

        model = nn.Sequential(*layers)

        self.assertEqual(
            model(torch.randn(1, 3, 128, 128)).size(), torch.Size([1, 4000, 4, 4])
        )

    def test_decoder_block(self):
        in_channels = 4000
        out_channels = 512
        kernel_size = 4
        stride = 2
        padding = 1

        layers = []

        for _ in range(4):
            layers.append(
                DecoderBlock(
                    in_channels=in_channels,
                    out_channels=out_channels,
                    kernel_size=kernel_size,
                    stride=stride,
                    padding=padding,
                )
            )
            in_channels = out_channels
            out_channels = in_channels // 2

        layers.append(
            nn.ConvTranspose2d(
                in_channels=in_channels,
                out_channels=3,
                kernel_size=kernel_size - 1,
                stride=stride // stride,
                padding=padding,
            )
        )

        model = nn.Sequential(*layers)

        self.assertEqual(
            model(torch.randn(1, 4000, 4, 4)).size(), torch.Size([1, 3, 64, 64])
        )

    def test_netG_size(self):
        self.assertEqual(
            self.netG(torch.randn(1, 3, 128, 128)).size(), torch.Size([1, 3, 64, 64])
        )

    def test_netD_size(self):
        self.assertEqual(
            self.netD(torch.randn(1, 3, 64, 64)).size(), torch.Size([1, 1, 8, 8])
        )

    def test_netG_total_params(self):
        self.assertEqual(Generator.total_params(self.netG), 40401059)

    def test_netD_total_params(self):
        self.assertEqual(Discriminator.total_params(self.netD), 1555585)

    def test_helpers(self):
        self.assertIsInstance(self.init["netG"], Generator)
        self.assertIsInstance(self.init["netD"], Discriminator)
        self.assertIsInstance(self.init["optimizerG"], optim.Adam)
        self.assertIsInstance(self.init["optimizerD"], optim.Adam)
        self.assertIsInstance(self.init["adversarial_loss"], AdversarialLoss)
        self.assertIsInstance(self.init["pixelwise_loss"], PixelLoss)


if __name__ == "__main__":
    unittest.main()

#### Trainer

In [ ]:
import os
import sys
import mlflow
import torch
import warnings
import numpy as np
import torch.nn as nn
import traceback
from tqdm import tqdm
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import StepLR
from torchvision.utils import save_image

warnings.filterwarnings("ignore")

sys.path.append("src/")

from dataloader import Loader
from helper import helpers
from utils import config, dump, load, device_init, weight_init, CustomException


class Trainer:
    def __init__(
        self,
        epochs=100,
        lr=0.0002,
        beta1=0.5,
        beta2=0.999,
        weight_decay=0.0001,
        momentum=0.9,
        adversarial_lambda=0.001,
        pixelwise_lamda=0.999,
        steps=10,
        step_size=10,
        gamma=0.1,
        device="cuda",
        adam=True,
        SGD=False,
        l1_regularization=False,
        l2_regularization=False,
        elasticnet_regularization=False,
        lr_scheduler=False,
        MLFlow=True,
        display=True,
        is_weight_init=True,
    ):
        self.epochs = epochs
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.weight_decay = weight_decay
        self.momentum = momentum
        self.device = device
        self.adam = adam
        self.SGD = SGD
        self.l1_regularization = l1_regularization
        self.l2_regularization = l2_regularization
        self.elasticnet_regularization = elasticnet_regularization
        self.lr_scheduler = lr_scheduler
        self.MLFlow = MLFlow
        self.display = display
        self.is_weight_init = is_weight_init
        self.adversarial_lambda = adversarial_lambda
        self.pixelwise_lamda = pixelwise_lamda
        self.steps = steps
        self.step_size = step_size
        self.gamma = gamma

        try:
            self.init = helpers(
                adam=self.adam,
                SGD=self.SGD,
                beta1=self.beta1,
                beta2=self.beta2,
                momentum=self.momentum,
                lr=self.lr,
            )

        except CustomException as e:
            print(e)
            traceback.print_exc()

        else:
            self.netG = self.init["netG"]
            self.netD = self.init["netD"]

            self.train_dataloader = self.init["train_dataloader"]
            self.valid_dataloader = self.init["valid_dataloader"]

            self.optimizerG = self.init["optimizerG"]
            self.optimizerD = self.init["optimizerD"]

            self.adversarial_loss = self.init["adversarial_loss"]
            self.pixelwise_loss = self.init["pixelwise_loss"]

            assert (
                self.init["train_dataloader"].__class__.__name__ == DataLoader.__name__
            )
            assert (
                self.init["valid_dataloader"].__class__.__name__ == DataLoader.__name__
            )

            assert self.init["netG"].__class__.__name__ == "Generator"
            assert self.init["netD"].__class__.__name__ == "Discriminator"

            assert self.init["adversarial_loss"].__class__.__name__ == "AdversarialLoss"
            assert self.init["pixelwise_loss"].__class__.__name__ == "PixelLoss"

            if self.is_weight_init:
                self.netG.apply(weight_init)
                self.netD.apply(weight_init)

            if self.lr_scheduler:
                self.schedulerG = StepLR(
                    optimizer=self.optimizerG,
                    step_size=self.step_size,
                    gamma=self.gamma,
                )
                self.schedulerD = StepLR(
                    optimizer=self.optimizerD,
                    step_size=self.step_size,
                    gamma=self.gamma,
                )

            self.device = device_init(device=device)

            self.netG.to(self.device)
            self.netD.to(self.device)

            self.loss = float("inf")

            self.total_netG_loss = []
            self.total_netD_loss = []
            self.history = {"netG_loss": [], "netD_loss": []}

    def l1_regularizer(self, model=None, value=0.01):
        if model is not None:
            return value * sum(torch.norm(params, 1) for params in model.parameters())
        else:
            raise CustomException(
                "Elastic net cannot be possible for regularization".capitalize()
            )

    def l2_regularizer(self, model=None, value=0.001):
        if model is not None:
            return value * sum(torch.norm(params, 2) for params in model.parameters())
        else:
            raise CustomException(
                "Elastic net cannot be possible for regularization".capitalize()
            )

    def elasticnet_regularizer(self, model=None, value=0.001):
        if model is not None:
            self.l1 = self.l1_regularization(model=model, value=value)
            self.l2 = self.l2_regularization(model=model, value=value)

            return value * (self.l1 + self.l2)

        else:
            raise CustomException(
                "Elastic net cannot be possible for regularization".capitalize()
            )

    def update_netG(self, **kwargs):
        self.optimizerG.zero_grad()

        X = kwargs["X"]
        y = kwargs["y"]

        generated_inpaint = self.netG(X)
        predicted_inpaint = self.netD(generated_inpaint)
        predicted_inpaint_loss = self.adversarial_loss(
            predicted_inpaint, torch.ones_like(predicted_inpaint)
        )

        pixelwise_loss = self.pixelwise_loss(generated_inpaint, y)

        total_netG_loss = (
            self.adversarial_lambda * predicted_inpaint_loss
            + self.pixelwise_lamda * pixelwise_loss
        )

        if self.l1_regularization:
            total_netG_loss += self.l1_regularizer(model=self.netG)

        if self.l2_regularization:
            total_netG_loss += self.l2_regularizer(model=self.netG)

        if self.elasticnet_regularization:
            total_netG_loss += self.elasticnet_regularizer(model=self.netG)

        total_netG_loss.backward()
        self.optimizerG.step()

        return total_netG_loss.item()

    def update_netD(self, **kwargs):
        self.optimizerD.zero_grad()

        X = kwargs["X"]
        y = kwargs["y"]

        generated_inpaint = self.netG(X)
        predicted_inpaint = self.netD(generated_inpaint)
        predicted_inpaint_loss = self.adversarial_loss(
            predicted_inpaint, torch.zeros_like(predicted_inpaint)
        )

        predicted_real = self.netD(y)
        predicted_real_loss = self.adversarial_loss(
            predicted_real, torch.ones_like(predicted_real)
        )

        total_netD_loss = 0.5 * (predicted_inpaint_loss + predicted_real_loss)

        if self.l1_regularization:
            total_netD_loss += self.l1_regularizer(model=self.netD)

        if self.l1_regularization:
            total_netD_loss += self.l2_regularizer(model=self.netD)

        if self.elasticnet_regularization:
            total_netD_loss += self.elasticnet_regularizer(model=self.netD)

        total_netD_loss.backward()
        self.optimizerD.step()

        return total_netD_loss.item()

    def show_progress(self, **kwargs):
        if self.display:
            print(
                "Epochs: [{}/{}] - netG_loss: [{:.4f}] - netD_loss: [{:.4f}]".format(
                    kwargs["epoch"],
                    self.epochs,
                    kwargs["netG_loss"],
                    kwargs["netD_loss"],
                )
            )
        else:
            print("Epochs: [{}/{}] is completed".format(kwargs["epoch"], self.epochs))

    def saved_checkpoints(self, **kwargs):
        train_models_path = config()["path"]["TRAIN_MODELS_PATH"]
        best_model_path = config()["path"]["BEST_MODEL_PATH"]

        netG_loss = kwargs["netG_loss"]
        epoch = kwargs["epoch"]

        if (not os.path.exists(train_models_path)) and (
            not os.path.exists(best_model_path)
        ):
            os.makedirs(train_models_path, exist_ok=True)
            os.makedirs(best_model_path, exist_ok=True)

        elif (os.path.exists(train_models_path)) and (os.path.exists(best_model_path)):
            if self.loss > netG_loss:
                self.loss = netG_loss

                torch.save(
                    {
                        "netG": self.netG.state_dict(),
                        "netG_loss": netG_loss,
                        "epoch": epoch,
                    },
                    os.path.join(best_model_path, "best_model.pth"),
                )

            torch.save(
                self.netG.state_dict(),
                os.path.join(train_models_path, "netG{}.pth".format(epoch)),
            )

        else:
            raise CustomException(
                "Cannot be saved the models in the checkpoints".capitalize()
            )

    def train(self):
        with mlflow.start_run(
            description="Context Encoders: Feature Learning by Inpainting Context Encoders for Inpainting & Self-Supervised Learning("
        ) as run:
            for epoch in tqdm(range(self.epochs)):
                self.netG_loss = []
                self.netD_loss = []

                for _, (X, y) in enumerate(self.train_dataloader):
                    X = X.to(self.device)
                    y = y.to(self.device)

                    self.netD_loss.append(self.update_netD(X=X, y=y))
                    self.netG_loss.append(self.update_netG(X=X, y=y))

                if self.lr_scheduler:
                    self.schedulerG.step()
                    self.schedulerD.step()

                self.show_progress(
                    netG_loss=np.mean(self.netG_loss),
                    netD_loss=np.mean(self.netD_loss),
                    epoch=epoch + 1,
                )

                try:
                    self.saved_checkpoints(
                        netG_loss=np.mean(self.netG_loss), epoch=epoch + 1
                    )
                except CustomException as e:
                    print(e)
                    traceback.print_exc()

                except Exception as e:
                    print(e)
                    traceback.print_exc()

                if (epoch + 1) % self.steps == 0:
                    X, y = next(iter(self.train_dataloader))
                    X = X.to(self.device)
                    y = y.to(self.device)

                    predicted_impaint = self.netG(X)

                    save_image(
                        predicted_impaint,
                        os.path.join(
                            config()["path"]["SAVE_IMAGE_PATH"],
                            "image{}.png".format(epoch + 1),
                        ),
                        nrow=1,
                    )

                self.history["netG_loss"].append(np.mean(self.netG_loss))
                self.history["netD_loss"].append(np.mean(self.netD_loss))

                mlflow.log_params(
                    {
                        "epochs": self.epochs,
                        "lr": self.lr,
                        "beta1": self.beta1,
                        "beta2": self.beta2,
                        "weight_decay": self.weight_decay,
                        "momentum": self.momentum,
                        "adversarial_lambda": self.adversarial_lambda,
                        "pixelwise_lamda": self.pixelwise_lamda,
                        "steps": self.steps,
                        "step_size": self.step_size,
                        "gamma": self.gamma,
                        "device": self.device,
                        "adam": self.adam,
                        "SGD": self.SGD,
                        "l1_regularization": self.l1_regularization,
                        "l2_regularization": self.l2_regularization,
                        "lr_scheduler": self.lr_scheduler,
                        "MLFlow": self.MLFlow,
                        "display": self.display,
                        "is_weight_init": self.is_weight_init,
                    }
                )

                mlflow.log_metric(
                    key="netG_loss", value=np.mean(self.netG_loss), step=epoch + 1
                )
                mlflow.log_metric(
                    key="netD_loss", value=np.mean(self.netD_loss), step=epoch + 1
                )

            dump(
                value=self.history,
                filename=os.path.join(
                    os.path.join(config()["path"]["METRCIS_PATH"], "history.pkl")
                ),
            )

            mlflow.pytorch.log_model(self.netG, "netG")
            mlflow.pytorch.log_model(self.netD, "netD")

            print(
                "Metrics in a pickle format is stored in the folder {}".format(
                    config()["path"]["METRCIS_PATH"]
                ).capitalize()
            )
            print("""mlflow is used, "run the command: mflow ui" """.capitalize())


if __name__ == "__main__":
    trainer = Trainer(
        epochs=100,
        lr=0.0002,
        beta1=0.5,
        beta2=0.999,
        weight_decay=0.0001,
        momentum=0.9,
        adversarial_lambda=0.001,
        pixelwise_lamda=0.999,
        steps=10,
        step_size=20,
        gamma=0.5,
        device="cuda",
        adam=True,
        SGD=False,
        l1_regularization=False,
        l2_regularization=False,
        lr_scheduler=False,
        MLFlow=True,
        display=True,
        is_weight_init=True,
    )

    trainer.train()

#### Tester

In [ ]:
class Tester:
    def __init__(self, model="best", device="cuda", dataloader="valid"):
        self.model = model
        self.device = device
        self.dataloader = dataloader

        self.device = device_init(device=device)

    def select_model(self):
        if self.model == "best":
            best_model_path = config()["path"]["BEST_MODEL_PATH"]

            if os.path.exists(best_model_path):
                model = os.path.join(best_model_path, "best_model.pth")
                model = torch.load(model)
                model = model["netG"]

                return model

            else:
                raise CustomException("Cannot be found the best model".capitalize())

    def select_dataloader(self):
        processed_path = config()["path"]["PROCESSED_DATA_PATH"]

        if self.dataloader == "train":
            if os.path.exists(processed_path):
                train_dataloader = load(
                    filename=os.path.join(processed_path, "train_dataloader.pkl")
                )
                return train_dataloader

            else:
                raise CustomException("Cannot be found the processed data".capitalize())

        else:
            if os.path.exists(processed_path):
                valid_dataloader = load(
                    filename=os.path.join(processed_path, "test_dataloader.pkl")
                )
                return valid_dataloader

            else:
                raise CustomException("Cannot be found the processed data".capitalize())

    def plot(self):
        try:
            self.netG = Generator()
            self.netG.load_state_dict(self.select_model())
            self.netG.to(self.device)

        except CustomException as e:
            print(e)
            traceback.print_exc()

        except Exception as e:
            print(e)
            traceback.print_exc()

        try:
            dataloader = self.select_dataloader()

            assert dataloader.__class__.__name__ == DataLoader.__name__

        except CustomException as e:
            print(e)
            traceback.print_exc()

        except Exception as e:
            print(e)
            traceback.print_exc()

        else:
            data, label = next(iter(dataloader))
            predicted = self.netG(data.to(self.device))

        number_of_rows = data.size(0) // 2
        number_of_columns = data.size(0) // number_of_rows

        plt.figure(figsize=(20, 15))

        for index, image in enumerate(predicted):
            predicted_inpaint = image.permute(1, 2, 0).cpu().detach().numpy()
            predicted_inpaint = (predicted_inpaint - predicted_inpaint.min()) / (
                predicted_inpaint.max() - predicted_inpaint.min()
            )

            X = data[index].permute(1, 2, 0).cpu().detach().numpy()
            X = (X - X.min()) / (X.max() - X.min())

            y = label[index].permute(1, 2, 0).cpu().detach().numpy()
            y = (y - y.min()) / (y.max() - y.min())

            plt.subplot(3 * number_of_rows, 3 * number_of_columns, 3 * index + 1)
            plt.imshow(X)
            plt.title("X")
            plt.axis("off")

            plt.subplot(3 * number_of_rows, 3 * number_of_columns, 3 * index + 2)
            plt.imshow(y)
            plt.title("y")
            plt.axis("off")

            plt.subplot(3 * number_of_rows, 3 * number_of_columns, 3 * index + 3)
            plt.imshow(predicted_inpaint)
            plt.title("pred")
            plt.axis("off")

        plt.tight_layout()
        plt.savefig(
            os.path.join(config()["path"]["SAVE_TEST_IMAGE_PATH"], "result.png")
        )
        plt.show()

        print(
            "Result image is saved in {}".format(
                os.path.join(config()["path"]["SAVE_TEST_IMAGE_PATH"], "result.png")
            )
        )


if __name__ == "__main__":

    test = Tester(model="best", device="cuda", dataloader="valid")

    test.plot()